# Projectile motion

In [2]:
# Import some libraries from python and set some defaults
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import sys
import os
import time
sys.path.append(os.getcwd())
# from scipy.interpolate import CubicSpline
import pickle
import copy
plt.rcParams.update({'font.size': 17})
params = {'axes.labelsize': 16, 'axes.titlesize': 16}
plt.rcParams.update(params)
rc('animation', html='html5')

In [3]:
# get the base file
import requests
from pathlib import Path
if Path("DEq_Solver.py").is_file():
    print("DEq_Solver already exist")
else:
    print("Downloading the DEq_solver")
    request = requests.get("https://raw.githubusercontent.com/keithonpy/Scientific_Computing/main/Base/DEq_Solver.py")
    with open("DEq_Solver.py", "wb") as f:
        f.write(request.content)
print("Deq_Solver imported")
from DEq_Solver import DEq_Solver

Deq_Solver imported


Implement the second and fourth order Runge-Kutta method in the cell below. You have access to 
- the current coordinates array as ``self.x``
- the current time as ``self.t``
- the derivative $dx/dt$ as ``self.kernel.dx_dt(self.x,self.t)``
- the timestep $\Delta t$ as ``self.delta_t``.

Note that both classes below are derived from the DEq_Solver class which is defined in DEq_Solver.py (same directory). They will thus feature all methods defined in DEq_Solver as well as the methods defined for the individual derived classes.


**2nd order Runge-Kutta method**

Remeber in Radioactive decay notebook, $dx/dt = f(x, t)$, the 2nd order Runge-Kuuta method improve the Euler's method:
$$
x' = x + f(x, t)\frac{\Delta t}{2} \\
x(t + \Delta t) = x(t) + f(x', t') \Delta t \\
x^{n+1} = x^{n} + f(x^{n} + \frac{\Delta t}{2} f(x^{n}, t^{n}), t^{n}+\frac{\Delta t}{2}) \Delta t
$$

**4th order Runge-Kutta method**
Further improvement of 2nd order Runge-Kutta method by taking more points
$$
x(t + \Delta t) = x(t) + \frac{\Delta t}{6} [f(x_{1}', t_{1}') + 2f(x_{2}',t_{2}')+ 2f(x_{3}', t_{3}')+f(x_{4}', t_{4}')]
$$

where
$$
x_{1}' = x \text{   &   } t_{1}' = t 
$$
$$
x_{2}' = x + f(x_{1}', t_{1}') \text{   &   } t_{2}' = t + \frac{\Delta t}{2} \\
$$
$$
x_{3}' = x + f(x_{2}', t_{2}') \text{   &   } t_{3}' = t + \frac{\Delta t}{2} \\
$$
$$
x_{4}' = x + f(x_{3}', t_{3}') \text{   &   } t_{4} = t + \Delta t
$$


In [21]:
class RK2Solver(DEq_Solver):
    def __init__(self,kernel):
        self.kernel  = kernel
    def makeStep(self):
        # implement the method for performing a step according to the second-order Runge-Kutta algorithm
        # You have access to
        # self.t: current time
        # self.delta_t: the timestep
        # self.x: state of the system at time t
        # self.kernel.dx_dt(x, t): the derivative, dx/dt, at time t
        # input: none
        # output: none
        # the routine should update x and t
        # YOUR CODE HERE
        # inner function
        x_pi = self.x + self.kernel.dx_dt(self.x, self.t)* self.delta_t / 2
        t_pi = self.t + self.delta_t / 2
        # outer function
        self.x += self.kernel.dx_dt(x_pi, t_pi)*self.delta_t 
        self.t += self.delta_t
        
class RK4Solver(DEq_Solver):
    def __init__(self,kernel):
        self.kernel  = kernel
    def makeStep(self):
        # implement the method for performing a step according to the fourth-order Runge-Kutta algorithm
        # You have access to
        # self.t: current time
        # self.delta_t: the timestep
        # self.x: state of the system at time t
        # self.kernel.dx_dt(x, t): the derivative, dx/dt, at time t
        # input: none
        # output: none
        # the routine should update x and t
        # YOUR CODE HERE
        x1 = self.x
        t1 = self.t
        x2 = self.x + self.kernel.dx_dt(x1, t1)*self.delta_t /2
        t2 = self.t + self.delta_t /2
        x3 = self.x + self.kernel.dx_dt(x2, t2)*self.delta_t/2
        t3 = self.t + self.delta_t /2
        x4 = self.x + self.kernel.dx_dt(x3, t3)*self.delta_t
        t4 = self.t + self.delta_t
        
        self.x += self.delta_t / 6 * (self.kernel.dx_dt(x1,t1)
                                      + 2*self.kernel.dx_dt(x2, t2) 
                                      + 2*self.kernel.dx_dt(x3, t3)
                                      + self.kernel.dx_dt(x4, t4))
        self.t += self.delta_t

In [22]:
class testKernel:
    def dx_dt(x,t):
        return 2.0*t+x

solver=RK2Solver(testKernel)
x0=np.array([0.0])
solver.initialise(x0,0,1,0.1)
for i in range(5):
    solver.makeStep()
assert np.isclose(solver.x[0],0.2948935)

In [23]:
class testKernel:
    def dx_dt(x,t):
        return 2.0*t+x[1]-3.0*x[0]

solver=RK4Solver(testKernel)
x0=np.array([1.0,0.2])
solver.initialise(x0,0,1,0.1)
for i in range(5):
    solver.makeStep()
assert np.isclose(solver.x,np.array([0.29898195, -0.50101805])).all()